In [1]:
import pip

package_names = ['awswrangler'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
#from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc
#from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#from utils import *
#from modeler import *
#subfolder = "data"
# os.listdir(subfolder)

from setup import *

////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
target :::::::::  target_desembolso_f2m_mayor_30_menor_180


In [3]:
# imports
from UTILITARIO_CODE.utils import targets
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.amazon.amazon_estimator import get_image_uri
import awswrangler as wr
import pandas as pd
import sagemaker
import boto3

print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)
print("sufijo: ", sufijo)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-09-27 18:58:45.326082
sufijo:  target_desembolso_f2m_mayor_30_menor_180_clasif_normal


In [4]:
print(len(seleccionadas))
print(seleccionadas)

19
['deuda_sf_prom_ult9m', 'saldo_coloc_direct_tc', 'prom_reprog_u12m', 'ult_var_saldo_ajustado_amt', 'nro_entid_financ_prom_ult9m_cnt', 'monto_adquirido_u6_amt', 'nro_var_10k_30k_negativa_u6', 'var_neta_saldo_ajustado_u3_amt', 'prom_gar_u12m', 'nroregs_reactiva_bcos', 'monto_pagado_u3_amt', 'saldo_coloc_direct_vig_bcos', 'saldo_coloc_direct_vig_cmpt', 'nro_entidades', 'monto_pagado_ult_rcc_amt', 'tendencia_nro_coloc_direct_bancos_v2', 'sow_cajas', 'prom_fae_u12m', 'nro_var_10k_30k_negativa_u3']


### Load data

In [5]:
print("uri_output: ", uri_output)
print("uri_contraste: ", uri_contraste)
print("uri_predict: ", uri_predict)

uri_output:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output3
uri_contraste:  s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste
uri_predict:  s3://sagemaker-us-east-1-058528764918/vpc/propension/prediccion


** OJO: modificar esta parte en caso de cambios de periodos**

In [6]:
#uri_validation_202102 = uri_contraste + '/202102_{}.csv'.format(sufijo)
uri_validation_202103 = uri_contraste + '/202103_{}.csv'.format(sufijo)
uri_validation_202104 = uri_contraste + '/202104_{}.csv'.format(sufijo)
uri_validation_202105 = uri_contraste + '/202105_{}.csv'.format(sufijo)
uri_validation_202106 = uri_contraste + '/202106_{}.csv'.format(sufijo)
uri_validation_202107 = uri_contraste + '/202107_{}.csv'.format(sufijo)

#print(uri_validation_202102)
print(uri_validation_202103)
print(uri_validation_202104)
print(uri_validation_202105)
print(uri_validation_202106)
print(uri_validation_202107)

s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202103_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202104_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202105_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202106_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202107_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv


In [7]:
#_2006 = wr.s3.read_csv(
#    uri_validation_202106, 
#    sep=',',
#    dtype={'cod_sbs_val': str}
#)

#_2006.head()

In [8]:
# _2006.shape

### FEEDBACK

In [9]:

campania_ibk = pd.read_csv(
    'resultados_preaprobados_aporbados_5.csv', 
    dtype={'PERIODO': str, 'NUM_RUC': str, 'COD_SBS': str, 'CODSBS': str, 'COD_UNICO': str}
)
campania_ibk = campania_ibk[campania_ibk['CAMPANHA'] == 'Nuevo Pre-aprobado']
campania_ibk.head()



,CODSBS,TIPDOC,NOMBRE_COMPLETO,TIPPERSONA,FLG_ACEPTA,FLG_GESTIONADO,FLG_DESEMBOLSADO,FLG_CONTACTADO,ID_EMPRESA,PERIODO,CAMPANHA,NUM_RUC,COD_UNICO,GESTION_TIPO_2,GESTION_SUBTIPO,GESTION_COMENTARIO,GESTION_FECHA_REGISTRO,GESTIONADO,Contactados,Acepta,Desembolsado,CTD_DESEMBOLSO,CTD_DESPLEGADOS,COLOCACION_NETA,FRESCURA_LEAD,OFERTA_APROBADA,flg_tlv,Acepta_EN,Acepta_tlv,Acepta_tlv_sin_contacto,BARRIDO,CE,INTERESADO,Pasa_pre_filtro,PASA_OFERTA,ENVIADO_GTP,TIPO_PERSONA,ID_CAMPANHA_INSTANCIA,canal,registro_ejecutivo,SUB_CAMPANHA,GESTION_TIPO_TLV,GESTION_TLV_NIVEL_1,GESTION_TLV_NIVEL_2,CANAL_INICIAL,FLG_REPROGRAMACION,LETTER,PERIODO_CAMPANHA,COD_SBS
1,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202106,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Fresco,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,56.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.TELEFONO ERRADO,A.TELEFONO ERRADO,NaN,NaN,10,202106,0120556819
2,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202103,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Repetido,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,28.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202103,0120556819
3,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,1,0,1,183875.0,202102,Nuevo Pre-aprobado,10004003573,0012854893,No Acepta Campaña,No necesita campaña,"Cliente no desea créditos, no dio mayor detall...",2021-02-19 16:04:00,GESTIONADO,contactado,NaN,Sin desembolso,0,1,NaN,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,24.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202102,0120556819
4,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202107,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Repetido,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,63.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.NO CONTACTO,A.NO CONTACTO,TLV,NaN,10,202107,0120556819
5,0072712960,1.0,TACO HUAYHUA NELIDA LUCY\r,1.0,0,0,0,0,1036710.0,202103,Nuevo Pre-aprobado,10004003913,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Fresco,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,28.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202103,0072712960


In [10]:
campania_ibk.isnull().sum()

CODSBS                         68
TIPDOC                         68
NOMBRE_COMPLETO               154
TIPPERSONA                     68
FLG_ACEPTA                      0
FLG_GESTIONADO                  0
FLG_DESEMBOLSADO                0
FLG_CONTACTADO                  0
ID_EMPRESA                     11
PERIODO                         0
CAMPANHA                        0
NUM_RUC                         0
COD_UNICO                  107832
GESTION_TIPO_2             156954
GESTION_SUBTIPO            157846
GESTION_COMENTARIO         159667
GESTION_FECHA_REGISTRO     156977
GESTIONADO                      0
Contactados                     0
Acepta                     200659
Desembolsado                    0
CTD_DESEMBOLSO                  0
CTD_DESPLEGADOS                 0
COLOCACION_NETA            201960
FRESCURA_LEAD                   0
OFERTA_APROBADA                 8
flg_tlv                    138724
Acepta_EN                  200670
Acepta_tlv                 139656
Acepta_tlv_sin

In [11]:
campania_ibk[campania_ibk['COD_SBS'].isnull()].groupby(by=['PERIODO']).size()

PERIODO
202103     1
202104    64
202107     1
202108     2
dtype: int64

In [12]:
print(campania_ibk.shape)
campania_ibk = campania_ibk[campania_ibk['COD_SBS'].notnull()]
campania_ibk['COLOCACION_NETA'] = campania_ibk['COLOCACION_NETA'].fillna(0)                        
print(campania_ibk.shape)

campania_ibk.head()

(203035, 49)
(202967, 49)


,CODSBS,TIPDOC,NOMBRE_COMPLETO,TIPPERSONA,FLG_ACEPTA,FLG_GESTIONADO,FLG_DESEMBOLSADO,FLG_CONTACTADO,ID_EMPRESA,PERIODO,CAMPANHA,NUM_RUC,COD_UNICO,GESTION_TIPO_2,GESTION_SUBTIPO,GESTION_COMENTARIO,GESTION_FECHA_REGISTRO,GESTIONADO,Contactados,Acepta,Desembolsado,CTD_DESEMBOLSO,CTD_DESPLEGADOS,COLOCACION_NETA,FRESCURA_LEAD,OFERTA_APROBADA,flg_tlv,Acepta_EN,Acepta_tlv,Acepta_tlv_sin_contacto,BARRIDO,CE,INTERESADO,Pasa_pre_filtro,PASA_OFERTA,ENVIADO_GTP,TIPO_PERSONA,ID_CAMPANHA_INSTANCIA,canal,registro_ejecutivo,SUB_CAMPANHA,GESTION_TIPO_TLV,GESTION_TLV_NIVEL_1,GESTION_TLV_NIVEL_2,CANAL_INICIAL,FLG_REPROGRAMACION,LETTER,PERIODO_CAMPANHA,COD_SBS
1,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202106,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Fresco,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,56.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.TELEFONO ERRADO,A.TELEFONO ERRADO,NaN,NaN,10,202106,0120556819
2,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202103,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Repetido,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,28.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202103,0120556819
3,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,1,0,1,183875.0,202102,Nuevo Pre-aprobado,10004003573,0012854893,No Acepta Campaña,No necesita campaña,"Cliente no desea créditos, no dio mayor detall...",2021-02-19 16:04:00,GESTIONADO,contactado,NaN,Sin desembolso,0,1,0.0,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,24.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202102,0120556819
4,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202107,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Repetido,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,63.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.NO CONTACTO,A.NO CONTACTO,TLV,NaN,10,202107,0120556819
5,0072712960,1.0,TACO HUAYHUA NELIDA LUCY\r,1.0,0,0,0,0,1036710.0,202103,Nuevo Pre-aprobado,10004003913,NaN,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Fresco,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,28.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202103,0072712960


In [13]:
from sklearn.metrics import roc_auc_score

In [14]:
def get_score_campania(mes_campania, data_rcc):
    camapania_target = campania_ibk[
        (campania_ibk['PERIODO'] == mes_campania)
    ]
    print(camapania_target.shape, camapania_target['PERIODO'].unique())
    
    try:
        print("-"*25)
        print("AUC BPE RCC:     ", roc_auc_score(data_rcc[target], data_rcc['score_desembolso']))
    except:
        pass
    
    camapania_target['COD_SBS'] = camapania_target['COD_SBS'].astype(int).astype(str)
    data_rcc['cod_sbs_val'] = data_rcc['cod_sbs_val'].astype(int).astype(str)
    
    validation_campania = data_rcc[data_rcc['cod_sbs_val'].isin(list(camapania_target['COD_SBS']))]
    try:
        roc_auc_campania = roc_auc_score(validation_campania[target], validation_campania['score_desembolso'])
        print("AUC CAMPANIA RCC: ", roc_auc_campania, roc_auc_campania*2-1)
    except:
        pass
    
    print("validation_campania: ", validation_campania.shape)
    validation_campania = validation_campania.merge(
        camapania_target, how='left', left_on='cod_sbs_val' , right_on ='COD_SBS'
    )
    print("camapania_target: ", camapania_target.shape)
    print("validation_campania: ", validation_campania.shape)
    
    roc_auc_acepta = roc_auc_score(validation_campania['FLG_ACEPTA'], validation_campania['score_desembolso'])
    print("AUC ACEPTA:      ",roc_auc_acepta, roc_auc_acepta*2-1)

    roc_auc_desembolsa = roc_auc_score(validation_campania['FLG_DESEMBOLSADO'], validation_campania['score_desembolso'])
    print("AUC DESEMBOLSA: ", roc_auc_desembolsa, roc_auc_desembolsa*2-1)
    
    group_ = validation_campania.groupby(by=['GRUPO_PROPENSION']).agg({
        target: ['count', 'sum', 'mean'], 
        'score_desembolso': 'mean',
        'FLG_ACEPTA': ['mean', 'sum'],
        'FLG_DESEMBOLSADO': ['mean', 'sum']
    })
    
    return group_, validation_campania

### 202102 ---> CAMPAÑA 202104

In [15]:
#_2002 = wr.s3.read_csv(
#    uri_validation_202102, 
#    sep=',',
#    dtype={'cod_sbs_val': str}
#)
#_2002.head(3)

In [16]:
#group_202104, campania_bpe_202104 = get_score_campania('202104', _2002)
#group_202104

In [17]:
#del _2002
#campania_bpe_202104['GRUPO_PROPENSION'].value_counts(normalize=True)*100

### 202103 ---> CAMPAÑA 202105

In [18]:
_2003 = wr.s3.read_csv(
    uri_validation_202103, 
    sep=',',
    dtype={'cod_sbs_val': str}
)
_2003.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION
0,48759904,0.0,6077.54,0.0,4558.153333,-616.17,1,0.0,0,-1794.37,0.00,0,1794.37,0.0,0.0,1,616.17,0.000000,1.0,0.000,0,0.007450,I
1,149898271,0.0,24525.75,0.0,17737.965000,900.00,1,900.0,1,-23266.88,2.25,0,24166.88,0.0,0.0,1,0.00,0.333333,1.0,0.000,1,0.030875,C
2,126112726,0.0,7000.35,0.0,0.000000,0.00,1,0.0,0,0.00,1.00,0,0.00,0.0,0.0,1,0.00,1.000000,0.0,5833.625,0,0.007600,I


In [19]:
group_202105, campania_bpe_202105 = get_score_campania('202105', _2003)
group_202105

(25748, 49) ['202105']
-------------------------
AUC BPE RCC:      0.7720062554415492
AUC CAMPANIA RCC:  0.6996912248591634 0.3993824497183267
validation_campania:  (24137, 23)
camapania_target:  (25748, 49)
validation_campania:  (24137, 72)
AUC ACEPTA:       0.6235948724177094 0.24718974483541878
AUC DESEMBOLSA:  0.6166512446839978 0.23330248936799558


target_desembolso_f2m_mayor_30_menor_180                  score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count    sum      mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                               
A                                                    2659  282.0  0.106055         0.089190   0.032343  86         0.018804  50
B                                                    2922  139.0  0.047570         0.047142   0.013005  38         0.007871  23
C                                                    2909  124.0  0.042626         0.033539   0.018563  54         0.007906  23
D                                                    2803  100.0  0.035676         0.025220   0.015341  43         0.008562  24
E                                                    3077   75.0  0.024374         0.020035   0.013650  42         0.007800  24
F                                                    3853   69.0  0.017908         0.015547   0.012458  48         0.007008  27
G                                                    2506   39.0  0.015563         0.012609   0.007582  19         0.004789  12
H                                                    2096   29.0  0.013836         0.010069   0.008111  17         0.006202  13
I                                                     885   11.0  0.012429         0.007794   0.002260   2         0.000000   0
J                                                     427    1.0  0.002342         0.005335   0.004684   2         0.000000   0

In [20]:
del _2003
campania_bpe_202105['GRUPO_PROPENSION'].value_counts(normalize=True)*100

NumExpr defaulting to 4 threads.


F    15.963044
E    12.748063
B    12.105896
C    12.052036
D    11.612876
A    11.016282
G    10.382400
H     8.683764
I     3.666570
J     1.769068
Name: GRUPO_PROPENSION, dtype: float64

### 202104 ---> CAMPAÑA 202106

In [21]:
_2004 = wr.s3.read_csv(
    uri_validation_202104, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2004.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION
0,203570678,0.0,80314.40,0.0,0.0000,-9379.89,0,250000.00,0,231724.84,0.000000,0,18275.16,231724.84,231724.84,1,9379.89,0.0,0.000,0.000000,0,0.031269,C
1,183894013,0.0,25441.24,0.0,0.0000,0.00,0,0.00,0,0.00,0.000000,1,0.00,0.00,0.00,1,0.00,1.0,0.000,0.000000,0,0.006905,I
2,74279325,0.0,99463.00,0.0,81768.9575,-1384.21,1,10225.52,0,-4265.70,0.833333,0,4265.70,8600.59,0.00,2,1384.21,1.0,0.883,18727.464167,0,0.033284,C


In [22]:
group_202106, campania_bpe_202106 = get_score_campania('202106', _2004)
group_202106

(20742, 49) ['202106']
-------------------------
AUC BPE RCC:      0.7526423472331193
AUC CAMPANIA RCC:  0.6720399926991016 0.3440799853982033
validation_campania:  (19680, 23)
camapania_target:  (20742, 49)
validation_campania:  (19680, 72)
AUC ACEPTA:       0.6138723519358686 0.2277447038717373
AUC DESEMBOLSA:  0.6044634869032645 0.20892697380652892


target_desembolso_f2m_mayor_30_menor_180                  score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count    sum      mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                               
A                                                    2122  193.0  0.090952         0.086145   0.028746  61         0.017436  37
B                                                    2400  139.0  0.057917         0.046631   0.020833  50         0.011667  28
C                                                    2221   97.0  0.043674         0.033200   0.020711  46         0.010356  23
D                                                    2363   67.0  0.028354         0.025023   0.014388  34         0.006348  15
E                                                    2799   74.0  0.026438         0.019945   0.016077  45         0.011433  32
F                                                    3007   62.0  0.020619         0.015466   0.012970  39         0.005653  17
G                                                    2132   51.0  0.023921         0.012621   0.011257  24         0.007036  15
H                                                    1372   24.0  0.017493         0.010026   0.010204  14         0.007289  10
I                                                     835    6.0  0.007186         0.007769   0.002395   2         0.002395   2
J                                                     429    2.0  0.004662         0.005341   0.000000   0         0.000000   0

In [23]:
del _2004
campania_bpe_202106['GRUPO_PROPENSION'].value_counts(normalize=True)*100

F    15.279472
E    14.222561
B    12.195122
D    12.007114
C    11.285569
G    10.833333
A    10.782520
H     6.971545
I     4.242886
J     2.179878
Name: GRUPO_PROPENSION, dtype: float64

### 202105 ---> CAMPAÑA 202107

In [24]:
_2005 = wr.s3.read_csv(
    uri_validation_202105, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2005.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION,GRUPO_PROPENSION_FINO
0,30419162,0.0,59487.23,0.0,0.000000,-2558.09,1,11789.22,0,7862.12,2.155599e+04,1,3927.10,43772.33,0.00,1,2558.09,1.000,0.000,0.0,0,0.033920,C,E
1,75300956,0.0,589446.28,0.0,328314.754167,-22543.07,2,633912.87,1,122979.29,1.162966e+06,0,510933.58,629297.28,629297.28,3,22543.07,0.875,0.000,0.0,1,0.077572,A,C
2,69166024,0.0,33044.68,0.0,15639.696667,-958.23,2,0.00,0,-3224.89,2.000000e+00,0,3224.89,28798.82,0.00,2,958.23,1.000,0.001,0.0,0,0.008842,I,I


In [25]:
group_202107, campania_bpe_202107 = get_score_campania('202107', _2005)
group_202107

(22544, 49) ['202107']
-------------------------
AUC BPE RCC:      0.76016894021159
AUC CAMPANIA RCC:  0.692504947151127 0.38500989430225396
validation_campania:  (21452, 24)
camapania_target:  (22544, 49)
validation_campania:  (21453, 73)
AUC ACEPTA:       0.5451926618404255 0.09038532368085095
AUC DESEMBOLSA:  0.5582470469194973 0.11649409383899467


target_desembolso_f2m_mayor_30_menor_180                  score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count    sum      mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                               
A                                                    1888  201.0  0.106462         0.088102   0.022246  42         0.010593  20
B                                                    2242  167.0  0.074487         0.048113   0.019179  43         0.009813  22
C                                                    2151  118.0  0.054858         0.034229   0.013482  29         0.006044  13
D                                                    2280   88.0  0.038596         0.025729   0.008772  20         0.005263  12
E                                                    2852   94.0  0.032959         0.020362   0.013324  38         0.004208  12
F                                                    3575   95.0  0.026573         0.015710   0.014266  51         0.007273  26
G                                                    2756   55.0  0.019956         0.013183   0.013062  36         0.006894  19
H                                                    1782   27.0  0.015152         0.010343   0.014590  26         0.007856  14
I                                                    1320   15.0  0.011364         0.007785   0.012879  17         0.002273   3
J                                                     607    3.0  0.004942         0.005570   0.003295   2         0.001647   1

### 202106 ---> CAMPAÑA 202108

In [26]:
_2006 = wr.s3.read_csv(
    uri_validation_202106, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2006.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION,GRUPO_PROPENSION_FINO
0,154769137,0.0,21749.62,0.00,1008.690000,-642.76,1,0.00,0,-1853.73,3.250,1,1853.73,8950.36,0.00,1,642.76,1.000000,0.000,0.0,0,0.005127,J,J
1,157683209,0.0,113613.00,85641.88,88991.732500,-2413.69,1,21148.22,0,7293.25,0.000,1,4374.28,91387.35,91387.35,1,2413.69,1.166667,0.000,0.0,0,0.016001,F,F
2,136905961,0.0,30885.53,0.00,6327.621667,11925.22,2,15233.98,0,11129.38,5448.325,1,4104.60,9445.35,0.00,2,854.97,1.000000,0.626,0.0,0,0.005851,J,J


In [27]:
group_202108, campania_bpe_202108 = get_score_campania('202108', _2006)
group_202108

(22611, 49) ['202108']
-------------------------
validation_campania:  (21770, 24)
camapania_target:  (22611, 49)
validation_campania:  (21770, 73)
AUC ACEPTA:       0.5331936959932949 0.06638739198658983
AUC DESEMBOLSA:  0.5595661064382577 0.11913221287651532


target_desembolso_f2m_mayor_30_menor_180           score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count  sum mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                        
A                                                    2402  0.0  0.0         0.091302   0.019567  47         0.012490  30
B                                                    2335  0.0  0.0         0.048035   0.014133  33         0.008137  19
C                                                    2085  0.0  0.0         0.034010   0.011511  24         0.008633  18
D                                                    2093  0.0  0.0         0.025412   0.010033  21         0.005256  11
E                                                    2879  0.0  0.0         0.020237   0.007294  21         0.004168  12
F                                                    3927  0.0  0.0         0.015539   0.011459  45         0.006112  24
G                                                    2768  0.0  0.0         0.013047   0.015535  43         0.007225  20
H                                                    1625  0.0  0.0         0.010211   0.009846  16         0.004923   8
I                                                    1184  0.0  0.0         0.007649   0.013514  16         0.008446  10
J                                                     472  0.0  0.0         0.005427   0.002119   1         0.002119   1

### 202107 ---> CAMPAÑA 202109

In [28]:
_2007 = wr.s3.read_csv(
    uri_validation_202107, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2007.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION,GRUPO_PROPENSION_FINO
0,67313267,0.0,182353.38,0.00,0.000000,0.00,1,0.00,0,0.00,0.0,1,0.00,0.01,0.01,1,0.00,1.0,0.000,0.0,0,0.015278,F,F
1,28947739,0.0,45091.08,0.00,7515.042500,-1701.57,1,21000.00,0,16968.41,0.0,0,4031.59,38195.57,0.00,2,1701.57,1.0,0.343,0.0,0,0.029214,D,E
2,53816525,0.0,842.23,16.99,723.640833,-2865.61,0,3501.56,0,-183.01,0.0,0,3020.67,16.99,16.99,1,2865.61,1.5,0.000,0.0,0,0.011504,H,H


In [29]:
group_202109, campania_bpe_202109 = get_score_campania('202109', _2007)
group_202109

(26256, 49) ['202109']
-------------------------
validation_campania:  (25331, 24)
camapania_target:  (26256, 49)
validation_campania:  (25331, 73)
AUC ACEPTA:       0.5145335256718491 0.029067051343698136
AUC DESEMBOLSA:  0.5663819725462191 0.13276394509243827


target_desembolso_f2m_mayor_30_menor_180           score_desembolso FLG_ACEPTA     FLG_DESEMBOLSADO    
                                                    count  sum mean             mean       mean sum             mean sum
GRUPO_PROPENSION                                                                                                        
A                                                    2631  0.0  0.0         0.094557   0.019384  51         0.008362  22
B                                                    2609  0.0  0.0         0.049739   0.018015  47         0.008816  23
C                                                    2412  0.0  0.0         0.035423   0.013682  33         0.004146  10
D                                                    2118  0.0  0.0         0.026762   0.011331  24         0.004721  10
E                                                    3225  0.0  0.0         0.020926   0.011163  36         0.002481   8
F                                                    3903  0.0  0.0         0.015976   0.012554  49         0.003843  15
G                                                    3737  0.0  0.0         0.013517   0.020337  76         0.005352  20
H                                                    1987  0.0  0.0         0.010536   0.009562  19         0.002516   5
I                                                    1783  0.0  0.0         0.007752   0.017386  31         0.003926   7
J                                                     926  0.0  0.0         0.005656   0.009719   9         0.005400   5

#### ENOQUE DE ENTRENAMIENTO ANTIGUO